# Sales Analysis for the whole year

## Get the DS from exel

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import datetime
pd.set_option('display.max_rows', 200)

### Read all the files and concatenete them to one DF
### Check if all the files are in

In [ ]:
files=[f for f in os.listdir("./Sales_data_monthly")]

all_dfs=[pd.read_csv("./Sales_data_monthly/"+f) for f in files]
df=pd.concat(all_dfs)

In [ ]:
sum([d.shape[0] for d in all_dfs])==df.shape[0]

### Save as a new csv file with info for the whole year. 

In [ ]:
#df.to_csv("Year sales.csv",index=False)

In [ ]:
sale_df=pd.read_csv("Sales_data_full_year.csv")

In [ ]:
sale_df.head()

### Drop the missing values

In [ ]:
sale_df.shape

In [ ]:
sale_df=sale_df.dropna()

In [ ]:
sale_df.shape

### Let's check if we have some wrong data types in columns where we must have only numeric values

In [ ]:
sale_df[["Quantity Ordered","Price Each","Order Date","Order ID"]].head()

In [ ]:
def dtype_check():
    for col in ["Quantity Ordered","Price Each","Order Date","Order ID"]:
        if sale_df[col].str.isalpha().any():
            print(f"Wrong alpha data types in {col}!!!")
        elif sale_df[col].str.istitle().any():
            print(f"Wrong title data types in {col}!!!")
        else:
            print(f"No wrong data types in {col}!")
dtype_check()

### Lets check what titled data we have inside and drop it

In [ ]:
sale_df[sale_df["Quantity Ordered"].str.istitle()==True]

In [ ]:
sale_df.drop(sale_df[sale_df["Quantity Ordered"].str.istitle()==True].index, inplace=True)

In [ ]:
dtype_check()

### As I have seen in the graps later, there is data for the first January of 2020. Let's drop it to analyse only the 2019

In [ ]:
sale_df.shape

In [ ]:
sale_df.drop(sale_df[sale_df["Order Date"].str[6:8]=="20"].index, inplace=True)
sale_df.shape

In [ ]:
sale_df[sale_df["Order Date"].str[6:8]=="20"].any()

# Now the data is clean and we can add new columns. First I will start with a total column.

In [ ]:
sale_df["Total"]=sale_df["Quantity Ordered"].astype(int)*sale_df["Price Each"].astype(float)

In [ ]:
sale_df.head()

# Add month column, by taking the month number from "Order date" column

In [ ]:
month_nums=sale_df["Order Date"].str[:2].astype(int)
set(month_nums)

### Getting month names to turn the month numbers into them

In [ ]:
import calendar
month_names=list(calendar.month_name)

In [ ]:
sale_df["Month"]=[month_names[i] for i in month_nums]

In [ ]:
sale_df["Month"].unique()

# Adding state column
### The dict is taken from https://gist.github.com/rogerallen/1583593

In [ ]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))


In [ ]:
abbrev_to_us_state["TX"]

### Once again taking the required information with string slicing

In [ ]:
state_abbr = [i[-8:-6] for i in sale_df["Purchase Address"]]

In [ ]:
sale_df["State"] = [abbrev_to_us_state[i] for i in state_abbr]

In [ ]:
sale_df.head()

# Adding the category column

In [ ]:
sale_df["Product"].unique()

### The number of product types is not that big, so we can easily split them in categories manually

In [ ]:
all_cats={
"Cabel & batteries":['USB-C Charging Cable','Lightning Charging Cable','AA Batteries (4-pack)','AAA Batteries (4-pack)'],
"Headphones":['Bose SoundSport Headphones','Wired Headphones','Apple Airpods Headphones'],
"Phones":['Google Phone','iPhone','Vareebadd Phone'],
"Laptops":['Macbook Pro Laptop','ThinkPad Laptop'],
"Monitors":['27in 4K Gaming Monitor','Flatscreen TV','27in FHD Monitor', '20in Monitor','34in Ultrawide Monitor'],
"Household":['LG Dryer','LG Washing Machine']
}


In [ ]:
sale_df["Category"]=[cat 
                    for product in list(sale_df["Product"]) 
                    for cat in list(all_cats.keys())  
                    if product in all_cats[cat]]

In [ ]:
sale_df.head()

# Saving the resulting data frame after reindexing the columns and changing the data type if required

In [ ]:
new_df=sale_df.reindex(columns=["Order ID", 
                                "Product", "Category", "Quantity Ordered", "Price Each","Total",
                                "Order Date", "Month",
                                "Purchase Address","State"])
new_df.head()

In [ ]:
new_df.dtypes


In [ ]:
new_df["Order Date"] = pd.to_datetime(new_df["Order Date"]) 

In [ ]:
new_df = new_df.astype({"Order ID":"int", 
                        "Quantity Ordered":"int", "Price Each":"float",
                        })

In [ ]:
new_df.dtypes

In [ ]:
#new_df.to_csv("Detailed_sales_table.csv")

# Now the DF is finished and we can start to analyse and visualize it with mathplotlib. The visuals in mathplotlib are very basic and the more advanced visualization is available as power BI graphical report. It can be found in the at the end of this notebook or at the folder as file.
# Analysing all sales per month

In [ ]:
revenue=[int(new_df[new_df["Month"]==i]["Total"].sum()) for i in new_df["Month"].unique()]

revenue

In [ ]:
revenue_to_month=dict(zip(new_df["Month"].unique(),revenue))

In [ ]:
revenue_to_month

In [ ]:
fig = plt.figure(figsize=(12,6))
font1={'family':'serif','size':20}
font2={'family':'serif','size':15}
plt.title('Month revenue',fontdict=font1)
plt.xlabel("Months",fontdict=font2)
plt.ylabel("Revenue",fontdict=font2)
plt.bar(range(len(new_df["Month"].unique())), revenue, tick_label=new_df["Month"].unique())
plt.savefig('Month_rev2.png',dpi=300,bbox_inches="tight")
plt.show

In [ ]:
#A part of year sales for each month
percent_to_month=[ms/(sum(revenue)/100) for ms in revenue]

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.title('Month revenue in percentage',fontdict=font1)
plt.xlabel("Months",fontdict=font2)
plt.ylabel("Revenue",fontdict=font2)
plt.bar(range(len(new_df["Month"].unique())), percent_to_month, tick_label=new_df["Month"].unique())
plt.show

In [ ]:
fig = plt.figure(figsize=(12,6))
font1={'family':'serif','size':25}
font2={'family':'serif','size':20}
plt.plot(new_df["Month"].unique(),revenue)
plt.title('Month revenue',fontdict=font1)
plt.xlabel("Months",fontdict=font2)
plt.ylabel("Revenue",fontdict=font2)
plt.show

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.pie(percent_to_month, labels=new_df["Month"].unique(),autopct='%1.1f%%',textprops={'fontsize': 14})
plt.title('Month sales in percentage',fontdict=font1)
plt.show()

### Let's create another small DF thst will present the revenue for each month. It will allow us to calculate the percetage growth over the year and to practice in creating relationship between tables in power BI

In [ ]:
sales_short_df=pd.DataFrame({"Months":revenue_to_month.keys(), "Revenue":revenue_to_month.values()})

In [ ]:
sales_short_df["Months"] = pd.to_datetime(sales_short_df["Months"], format="%B")
sales_short_df = sales_short_df.sort_values(by=["Months"])

In [ ]:
sales_short_df["Months"] = sales_short_df["Months"].dt.month_name()

In [ ]:
sales_short_df

In [ ]:
sales_short_df["Growth_rate"]=sales_short_df["Revenue"].pct_change()
sales_short_df.head()

In [ ]:
#sales_short_df.to_csv("Monthly_growth_rate.csv", index=False)

# Average bill

In [ ]:
new_df.head()

In [ ]:
#average bill sum of a revenue divided by number of bills
average_bill=int(new_df["Total"].sum() / len(new_df["Order ID"].unique()))
print(f"The average bill is {average_bill} dollars!")

# Analysing the sales by state

In [ ]:
state_revenue=[ new_df[new_df["State"]==i]["Total"].sum()  for i in new_df["State"].unique()]
state_to_revenue=dict(zip(new_df["State"].unique(),state_revenue))

In [ ]:
state_to_revenue

In [ ]:
fig = plt.figure(figsize=(8,8),dpi=100)
plt.pie(state_revenue,labels=new_df["State"].unique(),autopct='%1.1f%%',textprops={'fontsize': 14})
plt.title('Sales in each state in percentage',fontdict=font1)
#plt.savefig('State_sales_perc.png',dpi=300,bbox_inches="tight")
plt.show()

# Analysis by category

In [ ]:
revenue_to_cat=[new_df[new_df["Category"]==cat]["Total"].sum() for cat in all_cats.keys()]

In [ ]:
revenue_to_cat

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.pie(revenue_to_cat, labels=all_cats.keys(), autopct='%1.1f%%', textprops={'fontsize': 14})
plt.title('Revenue by category',fontdict=font1)
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.title('Revenue by category',fontdict=font1)
plt.xlabel("Category",fontdict=font2)
plt.ylabel("Revenue",fontdict=font2)
plt.bar(all_cats.keys(), revenue_to_cat)
plt.show

# Interractive visual report created in Power BI 

In [ ]:
pip install powerbiclient

In [11]:
from powerbiclient import Report, models

In [12]:
# Import the DeviceCodeLoginAuthentication class to authenticate against Power BI
from powerbiclient.authentication import DeviceCodeLoginAuthentication

# Initiate device authentication
device_auth = DeviceCodeLoginAuthentication()

Performing device flow authentication. Please follow the instructions below.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code CMJ8ESNFM to authenticate.

Device flow authentication successfully completed.
You are now logged in.


In [19]:
report_id = "e204579a-100a-4885-8873-f19cc233b52f"
group_id = "3abc646e-3e29-41ad-bb0d-169a9de7fb91"

In [20]:
report = Report(group_id=group_id,report_id=report_id, auth=device_auth)

In [21]:
report

Report()